In [32]:
from dotenv import load_dotenv
load_dotenv()
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import numpy as np
import json
import os
import markdown
import time
import pickle
import re

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.graph import END, MessageGraph
from langchain_community.tools import BraveSearch
from langgraph.prebuilt import ToolExecutor
from langchain_core.tools import tool

In [3]:
def strip_markdown(md):
    html = markdown.markdown(md)
    soup = BeautifulSoup(html, features='html.parser')
    return soup.get_text()

In [25]:
llm_cache = pickle.load(open('./backend/files/llm_cache.pkl', 'rb'))
api_cache = {}

In [5]:
pp_model = ChatOpenAI(temperature=0, 
                   base_url='https://api.perplexity.ai', 
                   api_key=os.environ['PERPLEXITY_API_KEY'],
                  model='llama-3-sonar-large-32k-online')

In [26]:
def generate_answer(model, messages):
    messages_list = tuple([(i['role'], i['content']) for i in messages])
    if (model.model_name, messages_list) in llm_cache:
        return llm_cache[(model.model_name, messages_list)]
    result = model.invoke(messages).content
    llm_cache[(model.model_name, messages_list)] = result
    return llm_cache[(model.model_name, messages_list)]

In [27]:
def handle_get_requests(url, params):
    params = tuple([(i,j) for i,j in params.items()])
    if (url, params) in api_cache:
        return api_cache[(url, params)]
    api_cache[(url, params)] = requests.get(url, params)
    time.sleep(1)
    return api_cache[(url, params)]

In [28]:
class RAWG:
    def __init__(self, api_key):
        self.ak = api_key
        self.games_url = 'https://api.rawg.io/api/games'
    def search_game(self, query):
        params = {
            'key': self.ak,
            'page': 1,           # Specify which page of results to return
            'page_size': 10,     # Specify the number of results per page
            'search': query,
            
        }
        response = handle_get_requests(self.games_url, params).content
        return json.loads(response)['results'][0]

In [29]:
rawg = RAWG(os.environ['RAWG_API_KEY'])

In [35]:
def recommend_games(model, query, rawg):

    messages = [        
        {'role': 'system', 'content': '''You are the best Game recommendation engine in the world. \ 
Given any type of query related to recommending video games you are supposed to provide a list of games that satisfy the user's request and also explain briefly for each game why. Return the results in a numbered list.'''},
        {'role': 'user', 'content': query+' Return the results in a numbered list and explain briefly why each is a good fit in 50 words.'}
    ]
    answer = strip_markdown(generate_answer(model, messages))
    answer = re.sub(r'\[\d+\]', '', answer)
    print(answer)
    game_list = []
    for i in answer.split('\n'):
        if i:
            i = i.split(': ')
            if ': '.join(i[:-1]):
                game_list.append([': '.join(i[:-1]), i[-1]])
                
    recommended_games_info = {}
    for i,j in game_list:
        cur_game_info = rawg.search_game(i)
        cur_game_info['why'] = j
        recommended_games_info[cur_game_info['name']] = cur_game_info
        
    return recommended_games_info

In [36]:
recommended_games = recommend_games(pp_model,"Recommend me games that really good combat mechanics and brilliant story telling", rawg)

Here are some games with excellent combat mechanics, along with a brief explanation of why each is a good fit:

Bayonetta 3: Offers a fluid and stylish combat system that rewards players with a sense of satisfaction and power.
Bloodborne: Provides a challenging and reactive combat system that favors speed, timing, and visceral action, making it feel more like a 3D action game than a traditional RPG.
Sekiro: Shadows Die Twice: Features a combat system that is fast, fluid, and rewards players who master its mechanics with a satisfying sense of accomplishment, offering a high level of freedom and experimentation.
Shadow of the Colossus: Offers a unique combat system that focuses on taking down massive bosses, requiring careful planning and execution, and providing an epic and intense experience.
Devil May Cry 5: Boasts a smooth and stylish combat system that lets players feel like a true demon-slaying badass, with a seamless flow of weapons, combos, and special moves.
God of War Ragnarök:

In [37]:
pickle.dump(llm_cache, open('llm_cache.pkl', 'wb'))
pickle.dump(api_cache, open('api_cache.pkl', 'wb'))